## Datasets

In [ ]:
from anki_language_deck_generator.dataset.german_word_list import GermanWordList

word_list = GermanWordList()

for row in word_list:
    print(row)
    break

## Text Generator

In [ ]:
import torch

from anki_language_deck_generator.LLM.vicuna_7b_v15 import Vicuna

SYSTEM_PROMPT = "As a language model assisting with German language learning, provide precise explanations, examples, and translations for various German words. Focus on delivering responses in a structured, concise format, without adding any extra information or notes."
QUERY_PROMPT = "Provide the singular and plural forms of the German noun 'das Wasser', with their translations in parentheses. Use 'Singular:' for the singular form with its article, and 'Plural:' for the plural form with its article."

with torch.cuda.amp.autocast(dtype=torch.bfloat16):
    with Vicuna(SYSTEM_PROMPT) as vicuna:
        print(vicuna.shoot(QUERY_PROMPT))

## Text to Speech

In [ ]:
import torch
from datetime import datetime

from anki_language_deck_generator.T2S.tts_v2 import TTSV2
from anki_language_deck_generator.helper import io as io_utils

TEMP_DIR = io_utils.create_temp_directory(f'text_to_speech')
T2S_OUTPUT_FILENAME = f"TTS_{datetime.now().strftime('%y%m%d-%H%M%S')}"
TEXT = "A curious fox named Finn discovered a hidden grove in the enchanted forest, where fireflies danced around a crystal pond, whispering tales of starlit secrets."

with TTSV2('en', TEMP_DIR) as tts:
    tts.shoot(TEXT, T2S_OUTPUT_FILENAME)

## Anki Deck

In [ ]:
from anki_language_deck_generator.anki.style.two_sentences_polyglot_puzzler import TwoSentencesPuzzlerStyle, TwoSentencesPuzzlerFields, TwoSentencesPuzzlerNote
from anki_language_deck_generator.anki.deck import AnkiDeck

from anki_language_deck_generator.helper import io as io_helper

deck_style = TwoSentencesPuzzlerStyle()
deck_fields = TwoSentencesPuzzlerFields()
deck = AnkiDeck(deck_name='example', style=deck_style, fields=deck_fields)
note = TwoSentencesPuzzlerNote({
    '1_pzl': 'Das Wetter heute ist .... schön. (Hint: An adverb to enhance \'schön\')',
    '1_fil': 'Das Wetter heute ist sehr schön.',
    '1_ans': 'sehr',
    '1_ans_trans': 'very',
    '1_trans': 'The weather today is very nice.',
    '1_pzl_vce': '', # Add the 1st sentence puzzle voice here
    '1_fil_vce': '', # Add the 1st sentence filled voice here
    '1_trans_vce': '', # Add the 1st sentence translation voice here
    '2_pzl': 'Ich möchte ..... Kaffee trinken. (Hint: An indefinite article for \'Kaffee\')',
    '2_fil': 'Ich möchte einen Kaffee trinken.',
    '2_ans': 'einen',
    '2_ans_trans': 'a',
    '2_trans': 'I would like to drink a coffee.',
    '2_pzl_vce': '', # Add the 2nd sentence puzzle voice here
    '2_fil_vce': '', # Add the 2nd sentence filled voice here
    '2_trans_vce': '', # Add the 2nd sentence translation voice here
    'expl_1': 'The word \'sehr\' enhances adjectives or adverbs, similar to \'very\' in English.',
    'expl_2': '\'Einen\' is the accusative form of the indefinite article \'ein\' used for masculine nouns like \'Kaffee\'.'
})

deck.add_note(note)

deck_path = deck.save([])
assert deck_path != None

n_deleted_files = io_helper.delete_file(deck_path)
assert n_deleted_files == 1